#### base_m_survey - https://borninbradford.github.io/datadict/bib/bib_baseline/

In [2]:
import sys

sys.path.append('/Users/samrelins/Documents/LIDA/dental_project/src/')

from baseline_data_prep import *
from education_data_prep import *
from ga_data_prep import *
import os
import pandas as pd
import plotly.express as px
from patsy import dmatrices
import statsmodels.api as sm

### Load and Merge Baseline Questionnaire Data With GA Data

In [1]:
# load baseline / ga data points of interest
cols_of_interest = ['entity_id', "BiBPregNumber", 'ben0nobenf', 'edu0fthede',
                    "fin0manfin", 'edu0mumede', 'hhd0marchb', "h2o0twathm",
                    'imd_2010_decile_nat', 'mbqlcasep5gp', "smk0regsmk"]
bib_dir = "/Users/samrelins/Documents/LIDA/dental_project/data/bib_data/"

baseline_data = return_merged_baseline_ga_df(bib_dir, cols_of_interest)
baseline_data.info()

NameError: name 'return_merged_baseline_ga_df' is not defined

Model GAs using full edrecs dataset - simple logistic regression model

In [29]:
from sklearn.model_selection import train_test_split

drop_cols = ["has_dental_ga", "entity_id"]

X_train, X_test, y_train, y_test = train_test_split(
    baseline_ga.drop(drop_cols, axis=1),
    baseline_ga.has_dental_ga,
    test_size=0.33,
    stratify=baseline_ga.has_dental_ga
)
baseline_train = X_train
baseline_train["has_dental_ga"] = y_train

baseline_test = X_test
baseline_test["has_dental_ga"] = y_test

In [30]:
features = baseline_train.drop("has_dental_ga", axis=1).columns
patsy_string = "has_dental_ga ~ "
patsy_string += " + ".join(features)
y, X = dmatrices(patsy_string, data=baseline_train, return_type="dataframe")

mod = sm.Logit(y, X)
log_reg = mod.fit()
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.258915
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:          has_dental_ga   No. Observations:                 7018
Model:                          Logit   Df Residuals:                     6996
Method:                           MLE   Df Model:                           21
Date:                Wed, 30 Jun 2021   Pseudo R-squ.:                 0.02889
Time:                        14:49:52   Log-Likelihood:                -1817.1
converged:                       True   LL-Null:                       -1871.1
Covariance Type:            nonrobust   LLR p-value:                 1.035e-13
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -2.1982      0.344     -6.395      0.000      -2.872      -1.525
be

In [31]:
preds = log_reg.predict(X).values
preds_df = pd.DataFrame(dict(
    y_hat = preds,
    y = y.has_dental_ga
))

px.box(preds_df, x="y", y="y_hat")


In [13]:
from sklearn.naive_bayes import CategoricalNB

nb_train_set = baseline_train
for col in nb_train_set:
    if nb_train_set[col].dtype.name == "category":
        nb_train_set[col] = nb_train_set[col].cat.codes
nb_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7578 entries, 9088 to 6501
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ben0nobenf             7578 non-null   int8   
 1   edu0fthede             7578 non-null   int8   
 2   edu0mumede             7578 non-null   int8   
 3   hhd0marchb             7578 non-null   int8   
 4   imd_2010_quintile_nat  7578 non-null   int8   
 5   mbqlcasep5gp           7578 non-null   int8   
 6   has_dental_ga          7578 non-null   float64
dtypes: float64(1), int8(6)
memory usage: 162.8 KB


In [14]:
clf = CategoricalNB()
clf.fit(nb_train_set.drop("has_dental_ga", axis=1),
        nb_train_set.has_dental_ga)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
clf.score(nb_train_set.drop("has_dental_ga", axis=1),
          nb_train_set.has_dental_ga)

0.9257059910266561

In [16]:
nb_test_set = baseline_test
for col in nb_test_set:
    if nb_test_set[col].dtype.name == "category":
        nb_test_set[col] = nb_test_set[col].cat.codes

pred_probas = clf.predict_proba(nb_test_set.drop("has_dental_ga", axis=1))[:,1]
preds = clf.predict(nb_test_set.drop("has_dental_ga", axis=1))
preds_df = pd.DataFrame(dict(
    y_hat_prob = pred_probas,
    y_hat = preds,
    y = nb_test_set.has_dental_ga
))

px.box(preds_df, x="y", y="y_hat_prob")

In [17]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(preds_df.y, preds_df.y_hat))

,0,1
0,3455,0
1,278,0


In [18]:
from sklearn.metrics import f1_score
f1_score(preds_df.y, preds_df.y_hat)

0.0

In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(preds_df.y, preds_df.y_hat)

0.5